针对直播中提到对于模型融合stacking和blending方法的使用

优点：可以直接使用xgb、lgb原生库进行建模，效率远高于sklearn接口下的建模

###  使用heamy模块进行模型在线融合

In [1]:
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import  seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)    #显示所有列
pd.set_option('display.max_rows', 30)    #显示所有行
pd.set_option('max_colwidth',100)    #设置value的显示长度为100，默认为50

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [2]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron,PassiveAggressiveRegressor
from sklearn.linear_model import SGDClassifier,SGDRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor


from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from catboost import CatBoostRegressor

from sklearn.metrics import roc_auc_score

In [3]:
# import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
"""
sns 相关设置
@return:
"""
# 声明使用 Seaborn 样式
sns.set()
# 有五种seaborn的绘图风格，它们分别是：darkgrid, whitegrid, dark, white, ticks。默认的主题是darkgrid。
sns.set_style("whitegrid")
# 有四个预置的环境，按大小从小到大排列分别为：paper, notebook, talk, poster。其中，notebook是默认的。
sns.set_context('talk')
# 中文字体设置-黑体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False
# 解决Seaborn中文显示问题并调整字体大小
sns.set(font='SimHei')

In [4]:
X_train = pd.read_csv(r'D:\MyData\chenpf8\TianChi_Data\01_Introduction to Financial risk control - Loan default prediction\0927\x_train.csv',
                   encoding='UTF-8', header=0)
X_test = pd.read_csv(r'D:\MyData\chenpf8\TianChi_Data\01_Introduction to Financial risk control - Loan default prediction\0927\x_test.csv',
                   encoding='UTF-8', header=0)
y_train = pd.read_csv(r'D:\MyData\chenpf8\TianChi_Data\01_Introduction to Financial risk control - Loan default prediction\0927\y_train.csv',
                   encoding='UTF-8', header=0)

In [5]:
y_train = np.array(y_train).reshape(612742)
y_train = pd.Series(y_train)
y_train

0         1
1         0
2         0
3         0
4         0
         ..
612737    1
612738    0
612739    1
612740    0
612741    0
Length: 612742, dtype: int64

In [47]:
type(y_train)

pandas.core.series.Series

In [48]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
type(data)

pandas.core.series.Series

In [6]:
from sklearn.model_selection import KFold
folds = 10
seed = 2020
kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

In [7]:
X_train['employmentLength'] = X_train['employmentLength'].fillna(X_train['employmentLength'].mode()[0])
X_test['employmentLength'] = X_test['employmentLength'].fillna(X_test['employmentLength'].mode()[0])

In [29]:
X_test.isnull().sum()

loanAmnt                0
term                    0
interestRate            0
installment             0
grade                   0
                       ..
n13                     0
n14                     0
issueDateDT             0
grade_target_mean       0
subGrade_target_mean    0
Length: 47, dtype: int64

In [27]:
y_train.isnull().sum()

0

###  模型融合

####  单模型交叉验证效果

LR模型

In [16]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model = LogisticRegression()
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("lr_scotrainre_list:{}".format(cv_scores))
print("lr_score_mean:{}".format(np.mean(cv_scores)))
print("lr_score_std:{}".format(np.std(cv_scores)))

************************************ 1 ************************************
[0.5000700118871391]
************************************ 2 ************************************
[0.5000700118871391, 0.5001326456700442]
************************************ 3 ************************************
[0.5000700118871391, 0.5001326456700442, 0.5001372613167547]
************************************ 4 ************************************
[0.5000700118871391, 0.5001326456700442, 0.5001372613167547, 0.5000361925369966]
************************************ 5 ************************************
[0.5000700118871391, 0.5001326456700442, 0.5001372613167547, 0.5000361925369966, 0.5002238875368424]
lr_scotrainre_list:[0.5000700118871391, 0.5001326456700442, 0.5001372613167547, 0.5000361925369966, 0.5002238875368424]
lr_score_mean:0.5001199997895553
lr_score_std:6.44561690149286e-05


In [ ]:
#  LR模型先不用   效果看起来不大好

# def lr_model(X_train, y_train, X_test, y_test=None):
#     X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2)
#     model = LogisticRegression()
#     model.fit(X_train_split, y_train_split)
#     val_pred = model.predict(X_val)
    

SVC模型

In [26]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model = svm.SVR(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, tol=0.001, cache_size=200, 
          verbose=False, max_iter=250)
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("svc_scotrainre_list:{}".format(cv_scores))
print("svc_score_mean:{}".format(np.mean(cv_scores)))
print("svc_score_std:{}".format(np.std(cv_scores)))


************************************ 1 ************************************
[0.5]
************************************ 2 ************************************
[0.5, 0.5]
************************************ 3 ************************************
[0.5, 0.5, 0.5]
************************************ 4 ************************************
[0.5, 0.5, 0.5, 0.5]
************************************ 5 ************************************
[0.5, 0.5, 0.5, 0.5, 0.5]
lr_scotrainre_list:[0.5, 0.5, 0.5, 0.5, 0.5]
lr_score_mean:0.5
lr_score_std:0.0


KNN模型

In [32]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', 
                        algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, 
                        n_jobs=None)
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("knn_scotrainre_list:{}".format(cv_scores))
print("knn_score_mean:{}".format(np.mean(cv_scores)))
print("knn_score_std:{}".format(np.std(cv_scores)))


************************************ 1 ************************************
[0.5396836473574543]
************************************ 2 ************************************
[0.5396836473574543, 0.539156587087484]
************************************ 3 ************************************
[0.5396836473574543, 0.539156587087484, 0.5431882229481506]
************************************ 4 ************************************
[0.5396836473574543, 0.539156587087484, 0.5431882229481506, 0.5407109311950096]
************************************ 5 ************************************
[0.5396836473574543, 0.539156587087484, 0.5431882229481506, 0.5407109311950096, 0.541878633738216]
lr_scotrainre_list:[0.5396836473574543, 0.539156587087484, 0.5431882229481506, 0.5407109311950096, 0.541878633738216]
lr_score_mean:0.540923604465263
lr_score_std:0.0014659912515217583


Linear SVC  线性SVC

In [33]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model = LinearSVC()
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("lsvc_scotrainre_list:{}".format(cv_scores))
print("lsvc_score_mean:{}".format(np.mean(cv_scores)))
print("lsvc_score_std:{}".format(np.std(cv_scores)))

************************************ 1 ************************************
[0.511065915604736]
************************************ 2 ************************************
[0.511065915604736, 0.5694727330657054]
************************************ 3 ************************************
[0.511065915604736, 0.5694727330657054, 0.5007793198430176]
************************************ 4 ************************************
[0.511065915604736, 0.5694727330657054, 0.5007793198430176, 0.5]
************************************ 5 ************************************
[0.511065915604736, 0.5694727330657054, 0.5007793198430176, 0.5, 0.5078087200502085]
lsvc_scotrainre_list:[0.511065915604736, 0.5694727330657054, 0.5007793198430176, 0.5, 0.5078087200502085]
lsvc_score_mean:0.5178253377127335
lsvc_score_std:0.02616021885674948


Stochastic Gradient Descent  随机梯度下降

In [34]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model = SGDRegressor()
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("SGDR_scotrainre_list:{}".format(cv_scores))
print("SGDR_score_mean:{}".format(np.mean(cv_scores)))
print("SGDR_score_std:{}".format(np.std(cv_scores)))

************************************ 1 ************************************
[0.5204768530764398]
************************************ 2 ************************************
[0.5204768530764398, 0.47944146439620694]
************************************ 3 ************************************
[0.5204768530764398, 0.47944146439620694, 0.5137669590947763]
************************************ 4 ************************************
[0.5204768530764398, 0.47944146439620694, 0.5137669590947763, 0.4693036906315997]
************************************ 5 ************************************
[0.5204768530764398, 0.47944146439620694, 0.5137669590947763, 0.4693036906315997, 0.5680423450639122]
SGDR_scotrainre_list:[0.5204768530764398, 0.47944146439620694, 0.5137669590947763, 0.4693036906315997, 0.5680423450639122]
SGDR_score_mean:0.5102062624525869
SGDR_score_std:0.034878845236576764


Decision Tree

In [35]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model = DecisionTreeRegressor()
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("DT_scotrainre_list:{}".format(cv_scores))
print("DT_score_mean:{}".format(np.mean(cv_scores)))
print("DT_score_std:{}".format(np.std(cv_scores)))

************************************ 1 ************************************
[0.5521535618296557]
************************************ 2 ************************************
[0.5521535618296557, 0.5537688117869518]
************************************ 3 ************************************
[0.5521535618296557, 0.5537688117869518, 0.5532360707562042]
************************************ 4 ************************************
[0.5521535618296557, 0.5537688117869518, 0.5532360707562042, 0.5530340589611564]
************************************ 5 ************************************
[0.5521535618296557, 0.5537688117869518, 0.5532360707562042, 0.5530340589611564, 0.5501565388086365]
DT_scotrainre_list:[0.5521535618296557, 0.5537688117869518, 0.5532360707562042, 0.5530340589611564, 0.5501565388086365]
DT_score_mean:0.5524698084285209
DT_score_std:0.0012683985642687357


GBDT

In [10]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model =  GradientBoostingRegressor(learning_rate=0.01,max_depth = 15, min_samples_leaf=10, min_samples_split=10,
                                     subsample = 0.6 )
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("GBDT_scotrainre_list:{}".format(cv_scores))
print("GBDT_score_mean:{}".format(np.mean(cv_scores)))
print("GBDT_score_std:{}".format(np.std(cv_scores)))

************************************ 1 ************************************


KeyboardInterrupt: 

###  GBDT  不能处理缺失值。。

####  下一步  对 GBDT 调参一下吧

In [28]:
def xgb_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.1)
    train_matrix = xgb.DMatrix(X_train_split , label=y_train_split)
    valid_matrix = xgb.DMatrix(X_val , label=y_val)
    test_matrix = xgb.DMatrix(X_test)

    params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'gamma': 4,
        'min_child_weight': 1.2,
        'max_depth': 5,
        'lambda': 10,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'colsample_bylevel': 0.24,
        'eta': 0.05,
        'tree_method': 'exact',
        'seed': 2020,
        'n_jobs': -1,
        "silent": True,
    }
    watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
    
    model = xgb.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
    """计算在验证集上的得分"""
    val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后xgboost单模型在验证集上的AUC：{}'.format(roc_auc))
    """对测试集进行预测"""
    test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit)
    
    return test_pred

In [29]:
"""建立模型：【模型参数：xgb-->鱼佬baseline，lgb --> 贝叶斯调参】"""
from sklearn import metrics
import xgboost as xgb
import lightgbm as lgb

# def xgb_model(X_train, y_train, X_test, y_test=None):
#     X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.1)
#     train_matrix = xgb.DMatrix(X_train_split , label=y_train_split)
#     valid_matrix = xgb.DMatrix(X_val , label=y_val)
#     test_matrix = xgb.DMatrix(X_test)

#     params = {
#         'booster': 'gbtree',
#         'objective': 'binary:logistic',
#         'eval_metric': 'auc',
#         'gamma': 1,
#         'min_child_weight': 1.5,
#         'max_depth': 5,
#         'lambda': 10,
#         'subsample': 0.7,
#         'colsample_bytree': 0.7,
#         'colsample_bylevel': 0.7,
#         'eta': 0.04,
#         'tree_method': 'exact',
#         'seed': 2020,
#         'n_jobs': -1,
#         "silent": True,
#     }
#     watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
    
#     model = xgb.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
#     """计算在验证集上的得分"""
#     val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
#     fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
#     roc_auc = metrics.auc(fpr, tpr)
#     print('调参后xgboost单模型在验证集上的AUC：{}'.format(roc_auc))
#     """对测试集进行预测"""
#     test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit)
    
#     return test_pred
    

def lgb_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.1)
    train_matrix = lgb.Dataset(X_train_split, label=y_train_split)
    valid_matrix = lgb.Dataset(X_val, label=y_val)
    
    # 调参后的最优参数
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.01,
        'min_child_weight': 0.43,
        'num_leaves': 40,
        'max_depth': 19,
        'feature_fraction': 0.70,
        'bagging_fraction': 0.96,
        'bagging_freq': 51,
        'min_data_in_leaf': 51,
        'min_split_gain': 0.38,
        'reg_alpha': 9,
        'reg_lambda': 10,
        'seed': 2020,
        'n_jobs':-1,
        'silent': True,
        'verbose': -1,
    }
    
    model = lgb.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], verbose_eval=500, early_stopping_rounds=500)
    """计算在验证集上的得分"""
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后lightgbm单模型在验证集上的AUC：{}'.format(roc_auc))
    """对测试集进行预测"""
    test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    return test_pred

In [ ]:
#  2020.10.05   10折交叉跑出来的最有 cat_model   参数取值
{'target': 0.7301774380344502,
 'params': {'depth': 2.3174615032112014,
  'l2_leaf_reg': 18.55111059409566,
  'od_wait': 50.12183338126141}}

In [30]:
def cat_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.1)

    
    # 调参后的最优参数
    params = {
#                 'n_estimators':5000,
                'learning_rate': 0.05,
                'eval_metric':'AUC',
                'bootstrap_type': 'Bernoulli',
                'l2_leaf_reg': 18,
                'random_seed':2020,
                'od_type': 'Iter',
                'od_wait':50,
                'depth': 2,
                'allow_writing_files': False
                #'colsample_bylevel':0.7,
                }
    
    model = CatBoostRegressor(iterations=20000,**params)
    model.fit(X_train_split,y_train_split,eval_set=(X_val, y_val),cat_features=[], use_best_model=True, verbose=500)
    """计算在验证集上的得分"""
    val_pred = model.predict(X_val)
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后cat单模型在验证集上的AUC：{}'.format(roc_auc))
    """对测试集进行预测"""
    test_pred = model.predict(X_test)
    return test_pred

In [59]:
#  暂时不用（10.04）
def gbdt_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.1)
    
    model =  GradientBoostingRegressor(learning_rate=0.01,max_depth = 15, min_samples_leaf=10, min_samples_split=10,
                                      subsample = 0.6 )
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后gbdt单模型在验证集上的AUC：{}'.format(roc_auc))
    """对测试集进行预测"""
    test_pred = model.predict(X_test)
    return test_pred

In [66]:
cv_scores = []
for i, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train.iloc[train_index], X_train.iloc[valid_index], y_train.iloc[valid_index]
    
    model =  GradientBoostingRegressor()
    model.fit(X_train_split, y_train_split)
    val_pred = model.predict(X_val)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

************************************ 1 ************************************


KeyboardInterrupt: 

In [ ]:
# learning_rate=0.1,max_depth = 5, min_samples_leaf=10, min_samples_split=10,
#                                       subsample = 0.6 

In [20]:
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [18]:
from sklearn.model_selection import cross_val_score

"""定义GBDT优化函数"""
def rf_cv_gbdt(max_depth, min_samples_leaf, min_samples_split,subsample):
    # 建立模型
    model_gbdt = GradientBoostingClassifier(learning_rate=0.1, max_features='sqrt',
                                  n_estimators=2000,
                                  max_depth = int(max_depth),subsample = round(subsample,2),
                                  min_samples_leaf = int(min_samples_leaf), min_samples_split = int(min_samples_split),
                                  random_state = 2020)
                                 
    
    val = cross_val_score(model_gbdt, X_train_split, y_train_split, cv=10, scoring='roc_auc').mean()
    
    return val


###  对GBDT调参  就是在做梦！！！

In [21]:

from bayes_opt import BayesianOptimization
"""定义优化参数"""
bayes_gbdt = BayesianOptimization(
            rf_cv_gbdt,
            {
        'max_depth':(1, 30),
        'min_samples_leaf':(1, 30),
        'min_samples_split':(1, 30),
        'subsample':(0,0.8)
            }
            )
 
"""开始优化"""
bayes_gbdt.maximize(n_iter=10)


|   iter    |  target   | max_depth | min_sa... | min_sa... | subsample |
-------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
n_estimators=2000,
learning_rate=0.01,
max_depth=15,
max_features='sqrt',
min_samples_leaf=10,
min_samples_split=10,
loss='ls',
random_state =42

####  基于模型层面的融合

In [31]:
from heamy.dataset import Dataset
from heamy.estimator import Classifier

model_dataset = Dataset(X_train=X_train, y_train=y_train, X_test=X_test)
model_xgb = Classifier(dataset=model_dataset, estimator=xgb_model, name='xgb', use_cache=False)
model_lgb = Classifier(dataset=model_dataset, estimator=lgb_model, name='lgb', use_cache=False)
model_cat = Classifier(dataset=model_dataset, estimator=cat_model, name='cat', use_cache=False)
# model_gbdt = Classifier(dataset=model_dataset, estimator=gbdt_model, name='gbdt', use_cache=False)

####  使用stacking 方法进行模型融合

In [32]:
from heamy.pipeline import ModelsPipeline

# pipeline = ModelsPipeline(model_xgb, model_lgb,model_cat,model_gbdt)
pipeline = ModelsPipeline(model_xgb, model_lgb,model_cat)
pipeline

In [29]:
# 构建第一层新特征，其中k默认是5，表示5折交叉验证，full_test=True，对全部训练集进行训练得到基学习器，然后用基学习器对测试集预测得到新特征
stack_ds = pipeline.stack(k=5, seed=111, full_test=True)

[21:36:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69673	eval-auc:0.69349
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73320	eval-auc:0.72439
[400]	train-auc:0.74252	eval-auc:0.72828
[600]	train-auc:0.74948	eval-auc:0.73002
[800]	train-auc:0.75521	eval-auc:0.73099
[1000]	train-auc:0.76053	eval-auc:0.73167
[1200]	train-auc:0.76523	eval-auc:0.73201
[1400]	train-auc:0.76964	eval-auc:0.73230
[1600]	train-auc:0.77403	eval-auc:0.73239
[1800]	train-auc:0.77825	eval-auc:0.73245
[2000]	train-auc:0.78216	eval-auc:0.73235
Stopping. Best iter

[4500]	training's auc: 0.747513	valid_1's auc: 0.72935
[5000]	training's auc: 0.748073	valid_1's auc: 0.729415
Early stopping, best iteration is:
[4719]	training's auc: 0.748073	valid_1's auc: 0.729415
调参后lightgbm单模型在验证集上的AUC：0.7294148522726138
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.723265	valid_1's auc: 0.72002
[1000]	training's auc: 0.729677	valid_1's auc: 0.724728
[1500]	training's auc: 0.733712	valid_1's auc: 0.727153
[2000]	training's auc: 0.736807	valid_1's auc: 0.728672
[2500]	training's auc: 0.739349	valid_1's auc: 0.729531
[3000]	training's auc: 0.741641	valid_1's auc: 0.730275
[3500]	training's auc: 0.743721	valid_1's auc: 0.73075
[4000]	training's auc: 0.745528	valid_1's auc: 0.731112
[4500]	training's auc: 0.747162	valid_1's auc: 0.731355
[5000]	training's auc: 0.747602	valid_1's auc: 0.731445
Early stopping, best iteration is:
[4646]	training's auc: 0.747601	valid_1's auc: 0.731446
调参后lightgbm单模型在验证集上的AUC：0.7314456743152784
Tr

In [58]:
# 构建第一层新特征，其中k默认是5，表示5折交叉验证，full_test=True，对全部训练集进行训练得到基学习器，然后用基学习器对测试集预测得到新特征
stack_ds = pipeline.stack(k=5, seed=111, full_test=True)

[15:08:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69559	eval-auc:0.69879
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73214	eval-auc:0.72893
[400]	train-auc:0.74156	eval-auc:0.73284
[600]	train-auc:0.74848	eval-auc:0.73446
[800]	train-auc:0.75423	eval-auc:0.73525
[1000]	train-auc:0.75958	eval-auc:0.73576
[1200]	train-auc:0.76437	eval-auc:0.73611
[1400]	train-auc:0.76899	eval-auc:0.73627
[1600]	train-auc:0.77349	eval-auc:0.73622
Stopping. Best iteration:
[1429]	train-auc:0.76968	eval-auc:0.73629

调参后xgboost单模型在验证集上的AUC：0.736294077

LightGBMError: Parameter num_leaves should be of type int, got "105.6"

In [27]:
# 构建第一层新特征，其中k默认是5，表示5折交叉验证，full_test=True，对全部训练集进行训练得到基学习器，然后用基学习器对测试集预测得到新特征
stack_ds = pipeline.stack(k=5, seed=111, full_test=True)

0:	test: 0.6885987	best: 0.6885987 (0)	total: 87.9ms	remaining: 29m 17s
500:	test: 0.7274243	best: 0.7274243 (500)	total: 18.7s	remaining: 12m 7s
1000:	test: 0.7299934	best: 0.7299934 (1000)	total: 37.7s	remaining: 11m 56s
1500:	test: 0.7311493	best: 0.7311493 (1500)	total: 57.4s	remaining: 11m 47s
2000:	test: 0.7317701	best: 0.7317746 (1997)	total: 1m 18s	remaining: 11m 44s
2500:	test: 0.7322049	best: 0.7322049 (2500)	total: 1m 39s	remaining: 11m 37s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7323558194
bestIteration = 2628

Shrink model to first 2629 iterations.
调参后cat单模型在验证集上的AUC：0.7323558193605897
0:	test: 0.6896257	best: 0.6896257 (0)	total: 40.9ms	remaining: 13m 37s
500:	test: 0.7283696	best: 0.7283696 (500)	total: 22.1s	remaining: 14m 21s


KeyboardInterrupt: 

In [52]:
#  2020.09.30 
stack_ds = pipeline.stack(k=5, seed=111, full_test=True)

[12:32:40] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69670	eval-auc:0.69445
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73277	eval-auc:0.72471
[400]	train-auc:0.74234	eval-auc:0.72850
[600]	train-auc:0.74904	eval-auc:0.72994
[800]	train-auc:0.75467	eval-auc:0.73075
[1000]	train-auc:0.75997	eval-auc:0.73131
[1200]	train-auc:0.76494	eval-auc:0.73162
[1400]	train-auc:0.76951	eval-auc:0.73189
[1600]	train-auc:0.77396	eval-auc:0.73191
Stopping. Best iteration:
[1418]	train-auc:0.76995	eval-auc:0.73194

调参后xgboost单模型在验证集上的AUC：0.731938888

[5000]	training's auc: 0.747609	valid_1's auc: 0.733524
Early stopping, best iteration is:
[4842]	training's auc: 0.747608	valid_1's auc: 0.733525
调参后lightgbm单模型在验证集上的AUC：0.733524622050751
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.722917	valid_1's auc: 0.717746
[1000]	training's auc: 0.729871	valid_1's auc: 0.722777
[1500]	training's auc: 0.733873	valid_1's auc: 0.725099
[2000]	training's auc: 0.736885	valid_1's auc: 0.726627
[2500]	training's auc: 0.739339	valid_1's auc: 0.727637
[3000]	training's auc: 0.741559	valid_1's auc: 0.728321
[3500]	training's auc: 0.743659	valid_1's auc: 0.729043
[4000]	training's auc: 0.745451	valid_1's auc: 0.729464
[4500]	training's auc: 0.747128	valid_1's auc: 0.72991
[5000]	training's auc: 0.748096	valid_1's auc: 0.730133
Early stopping, best iteration is:
[4866]	training's auc: 0.748095	valid_1's auc: 0.730133
调参后lightgbm单模型在验证集上的AUC：0.7301327446002972
Training until validation scores don't improve for 500 ro

In [22]:
#  2020.09.30   晚上
stack_ds = pipeline.stack(k=5, seed=111, full_test=True)

[22:56:46] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69576	eval-auc:0.69581
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73255	eval-auc:0.72574
[400]	train-auc:0.74214	eval-auc:0.72954
[600]	train-auc:0.74887	eval-auc:0.73120
[800]	train-auc:0.75474	eval-auc:0.73235
[1000]	train-auc:0.75983	eval-auc:0.73283
[1200]	train-auc:0.76454	eval-auc:0.73320
[1400]	train-auc:0.76909	eval-auc:0.73348
[1600]	train-auc:0.77354	eval-auc:0.73366
[1800]	train-auc:0.77769	eval-auc:0.73365
Stopping. Best iteration:
[1679]	train-auc:0.77526	eval-auc:0

[1500]	training's auc: 0.748995	valid_1's auc: 0.731073
[2000]	training's auc: 0.754026	valid_1's auc: 0.731683
[2500]	training's auc: 0.758722	valid_1's auc: 0.732186
[3000]	training's auc: 0.7629	valid_1's auc: 0.732485
[3500]	training's auc: 0.767051	valid_1's auc: 0.732623
[4000]	training's auc: 0.770822	valid_1's auc: 0.732753
[4500]	training's auc: 0.774325	valid_1's auc: 0.732863
[5000]	training's auc: 0.777463	valid_1's auc: 0.732835
Early stopping, best iteration is:
[4672]	training's auc: 0.775455	valid_1's auc: 0.732887
调参后lightgbm单模型在验证集上的AUC：0.7328869542379927
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.732012	valid_1's auc: 0.724091
[1000]	training's auc: 0.742646	valid_1's auc: 0.72891
[1500]	training's auc: 0.749129	valid_1's auc: 0.730175
[2000]	training's auc: 0.754474	valid_1's auc: 0.730795
[2500]	training's auc: 0.759185	valid_1's auc: 0.731129
[3000]	training's auc: 0.763277	valid_1's auc: 0.731315
[3500]	training's auc: 0

In [14]:
#  2020.10.04   下午
stack_ds = pipeline.stack(k=10, seed=111, full_test=True)

[15:29:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69594	eval-auc:0.69681
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73099	eval-auc:0.72701
[400]	train-auc:0.73968	eval-auc:0.73085
[600]	train-auc:0.74562	eval-auc:0.73257
[800]	train-auc:0.75076	eval-auc:0.73355
[1000]	train-auc:0.75500	eval-auc:0.73399
[1200]	train-auc:0.75912	eval-auc:0.73438
[1400]	train-auc:0.76300	eval-auc:0.73460
[1600]	train-auc:0.76672	eval-auc:0.73471
[1800]	train-auc:0.77024	eval-auc:0.73474
Stopping. Best iteration:
[1765]	train-auc:0.76959	eval-auc:0

[800]	train-auc:0.75079	eval-auc:0.72784
[1000]	train-auc:0.75531	eval-auc:0.72867
[1200]	train-auc:0.75952	eval-auc:0.72914
[1400]	train-auc:0.76333	eval-auc:0.72946
[1600]	train-auc:0.76698	eval-auc:0.72955
[1800]	train-auc:0.77046	eval-auc:0.72966
[2000]	train-auc:0.77376	eval-auc:0.72966
Stopping. Best iteration:
[1938]	train-auc:0.77272	eval-auc:0.72974

调参后xgboost单模型在验证集上的AUC：0.7297353771949726
[17:04:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69561	eval-auc:0.69497
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73059	eval-au

[4000]	training's auc: 0.764916	valid_1's auc: 0.73552
[4500]	training's auc: 0.768126	valid_1's auc: 0.735563
[5000]	training's auc: 0.77124	valid_1's auc: 0.735674
[5500]	training's auc: 0.774041	valid_1's auc: 0.735808
[6000]	training's auc: 0.776649	valid_1's auc: 0.735823
Early stopping, best iteration is:
[5664]	training's auc: 0.774863	valid_1's auc: 0.735881
调参后lightgbm单模型在验证集上的AUC：0.7358807279578148
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.730988	valid_1's auc: 0.723078
[1000]	training's auc: 0.740602	valid_1's auc: 0.728302
[1500]	training's auc: 0.746476	valid_1's auc: 0.730041
[2000]	training's auc: 0.7509	valid_1's auc: 0.730721
[2500]	training's auc: 0.754929	valid_1's auc: 0.73119
[3000]	training's auc: 0.75856	valid_1's auc: 0.731428
[3500]	training's auc: 0.762185	valid_1's auc: 0.731791
[4000]	training's auc: 0.765385	valid_1's auc: 0.731872
[4500]	training's auc: 0.768381	valid_1's auc: 0.732007
[5000]	training's auc: 0.77

0:	test: 0.6931518	best: 0.6931518 (0)	total: 59.4ms	remaining: 19m 48s
500:	test: 0.7319833	best: 0.7319852 (497)	total: 25.2s	remaining: 16m 19s
1000:	test: 0.7345765	best: 0.7345765 (1000)	total: 49.1s	remaining: 15m 32s
1500:	test: 0.7355817	best: 0.7355817 (1500)	total: 1m 13s	remaining: 15m 2s
2000:	test: 0.7362601	best: 0.7362601 (2000)	total: 1m 37s	remaining: 14m 33s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7366145785
bestIteration = 2270

Shrink model to first 2271 iterations.
调参后cat单模型在验证集上的AUC：0.736614578461191
0:	test: 0.6899988	best: 0.6899988 (0)	total: 48.2ms	remaining: 16m 3s
500:	test: 0.7279008	best: 0.7279008 (500)	total: 24.7s	remaining: 16m
1000:	test: 0.7302606	best: 0.7302606 (1000)	total: 48.9s	remaining: 15m 27s
1500:	test: 0.7312769	best: 0.7312769 (1500)	total: 1m 12s	remaining: 14m 55s
2000:	test: 0.7318255	best: 0.7318413 (1988)	total: 1m 36s	remaining: 14m 27s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7

In [12]:
#  2020.10.04   晚上
stack_ds = pipeline.stack(k=10, seed=111, full_test=True)

[23:31:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69664	eval-auc:0.69102
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73089	eval-auc:0.72377
[400]	train-auc:0.73982	eval-auc:0.72853
[600]	train-auc:0.74593	eval-auc:0.73034
[800]	train-auc:0.75112	eval-auc:0.73156
[1000]	train-auc:0.75557	eval-auc:0.73207
[1200]	train-auc:0.75978	eval-auc:0.73235
[1400]	train-auc:0.76373	eval-auc:0.73266
[1600]	train-auc:0.76732	eval-auc:0.73279
[1800]	train-auc:0.77091	eval-auc:0.73296
[2000]	train-auc:0.77424	eval-auc:0.73287
Stopping. Best iter

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73044	eval-auc:0.72629
[400]	train-auc:0.73951	eval-auc:0.73053
[600]	train-auc:0.74527	eval-auc:0.73207
[800]	train-auc:0.75028	eval-auc:0.73293
[1000]	train-auc:0.75479	eval-auc:0.73331
[1200]	train-auc:0.75880	eval-auc:0.73367
[1400]	train-auc:0.76269	eval-auc:0.73408
[1600]	train-auc:0.76640	eval-auc:0.73427
[1800]	train-auc:0.77000	eval-auc:0.73443
[2000]	train-auc:0.77331	eval-auc:0.73442
Stopping. Best iteration:
[1980]	train-auc:0.77303	eval-auc:0.73448

调参后xgboost单模型在验证集上的AUC：0.7344792575348662
[01:09:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69617	eval-auc:0

[4500]	training's auc: 0.76823	valid_1's auc: 0.733601
[5000]	training's auc: 0.771421	valid_1's auc: 0.733592
Early stopping, best iteration is:
[4690]	training's auc: 0.769395	valid_1's auc: 0.733632
调参后lightgbm单模型在验证集上的AUC：0.7336324388474981
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.731132	valid_1's auc: 0.72288
[1000]	training's auc: 0.740499	valid_1's auc: 0.72827
[1500]	training's auc: 0.746301	valid_1's auc: 0.730136
[2000]	training's auc: 0.750766	valid_1's auc: 0.730976
[2500]	training's auc: 0.754675	valid_1's auc: 0.731471
[3000]	training's auc: 0.758361	valid_1's auc: 0.731881
[3500]	training's auc: 0.761847	valid_1's auc: 0.732009
[4000]	training's auc: 0.765124	valid_1's auc: 0.732155
[4500]	training's auc: 0.768356	valid_1's auc: 0.732352
[5000]	training's auc: 0.771447	valid_1's auc: 0.732451
[5500]	training's auc: 0.774391	valid_1's auc: 0.732443
Early stopping, best iteration is:
[5179]	training's auc: 0.772569	valid_1's auc

1500:	test: 0.7312047	best: 0.7312047 (1500)	total: 1m 12s	remaining: 14m 55s
2000:	test: 0.7317653	best: 0.7317689 (1996)	total: 1m 36s	remaining: 14m 25s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7317988242
bestIteration = 2022

Shrink model to first 2023 iterations.
调参后cat单模型在验证集上的AUC：0.7317988241615572
0:	test: 0.6916879	best: 0.6916879 (0)	total: 45.5ms	remaining: 15m 10s
500:	test: 0.7305642	best: 0.7305642 (500)	total: 24.7s	remaining: 16m 1s
1000:	test: 0.7334131	best: 0.7334131 (1000)	total: 48.8s	remaining: 15m 26s
1500:	test: 0.7347390	best: 0.7347390 (1500)	total: 1m 12s	remaining: 14m 57s
2000:	test: 0.7353876	best: 0.7353963 (1992)	total: 1m 36s	remaining: 14m 31s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7356750763
bestIteration = 2297

Shrink model to first 2298 iterations.
调参后cat单模型在验证集上的AUC：0.7356750763408942
0:	test: 0.6914698	best: 0.6914698 (0)	total: 47.9ms	remaining: 15m 57s
500:	test: 0.7257763	best: 0.7257763 

In [33]:
#  2020.10.07   下午
stack_ds = pipeline.stack(k=10, seed=111, full_test=True)

[15:54:02] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69368	eval-auc:0.69303
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73082	eval-auc:0.72691
[400]	train-auc:0.73852	eval-auc:0.73094
[600]	train-auc:0.74356	eval-auc:0.73268
[800]	train-auc:0.74777	eval-auc:0.73376
[1000]	train-auc:0.75136	eval-auc:0.73468
[1200]	train-auc:0.75431	eval-auc:0.73504
[1400]	train-auc:0.75716	eval-auc:0.73531
[1600]	train-auc:0.75985	eval-auc:0.73545
[1800]	train-auc:0.76245	eval-auc:0.73566
[2000]	train-auc:0.76483	eval-auc:0.73561
[2200]	train-auc:0.

[200]	train-auc:0.73012	eval-auc:0.72603
[400]	train-auc:0.73813	eval-auc:0.72965
[600]	train-auc:0.74322	eval-auc:0.73116
[800]	train-auc:0.74718	eval-auc:0.73205
[1000]	train-auc:0.75072	eval-auc:0.73255
[1200]	train-auc:0.75396	eval-auc:0.73291
[1400]	train-auc:0.75684	eval-auc:0.73324
[1600]	train-auc:0.75950	eval-auc:0.73348
[1800]	train-auc:0.76202	eval-auc:0.73361
[2000]	train-auc:0.76442	eval-auc:0.73367
[2200]	train-auc:0.76684	eval-auc:0.73367
Stopping. Best iteration:
[2058]	train-auc:0.76512	eval-auc:0.73376

调参后xgboost单模型在验证集上的AUC：0.7337622343239985
[16:58:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69353	eval-auc:0.69519
Multiple

Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.730331	valid_1's auc: 0.729939
[1000]	training's auc: 0.739808	valid_1's auc: 0.735213
[1500]	training's auc: 0.745463	valid_1's auc: 0.737102
[2000]	training's auc: 0.749955	valid_1's auc: 0.73802
[2500]	training's auc: 0.753929	valid_1's auc: 0.738561
[3000]	training's auc: 0.757652	valid_1's auc: 0.738972
[3500]	training's auc: 0.761228	valid_1's auc: 0.739274
[4000]	training's auc: 0.764524	valid_1's auc: 0.739444
[4500]	training's auc: 0.767798	valid_1's auc: 0.739668
[5000]	training's auc: 0.770878	valid_1's auc: 0.739794
[5500]	training's auc: 0.773611	valid_1's auc: 0.739879
[6000]	training's auc: 0.776178	valid_1's auc: 0.739976
[6500]	training's auc: 0.778627	valid_1's auc: 0.740022
[7000]	training's auc: 0.780785	valid_1's auc: 0.740093
[7500]	training's auc: 0.782958	valid_1's auc: 0.739982
Early stopping, best iteration is:
[7094]	training's auc: 0.781245	valid_1's auc: 0.740111
调参后lightg

4500:	test: 0.7263862	best: 0.7263862 (4500)	total: 2m 30s	remaining: 8m 38s
5000:	test: 0.7266943	best: 0.7266959 (4996)	total: 2m 47s	remaining: 8m 21s
5500:	test: 0.7269326	best: 0.7269443 (5462)	total: 3m 3s	remaining: 8m 4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7269442822
bestIteration = 5462

Shrink model to first 5463 iterations.
调参后cat单模型在验证集上的AUC：0.7269442821677845
0:	test: 0.6564819	best: 0.6564819 (0)	total: 38.8ms	remaining: 12m 56s
500:	test: 0.7247015	best: 0.7247015 (500)	total: 17.2s	remaining: 11m 9s
1000:	test: 0.7273281	best: 0.7273281 (1000)	total: 34.1s	remaining: 10m 47s
1500:	test: 0.7285667	best: 0.7285667 (1500)	total: 50.9s	remaining: 10m 27s
2000:	test: 0.7293905	best: 0.7293936 (1998)	total: 1m 7s	remaining: 10m 8s
2500:	test: 0.7300751	best: 0.7300812 (2498)	total: 1m 24s	remaining: 9m 48s
3000:	test: 0.7306657	best: 0.7306657 (3000)	total: 1m 40s	remaining: 9m 31s
3500:	test: 0.7310676	best: 0.7310683 (3499)	total: 1m 57s	rema

In [34]:
#  2020.10.07  晚上1

from sklearn.linear_model import LogisticRegression
# 第二层使用逻辑回归进行stack
LogisticRegression(solver='lbfgs')
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
# 测试集的预测结果
test_pred = stacker.predict()
test_pred

array([0.08364356, 0.33550796, 0.6041077 , ..., 0.18608774, 0.21779624,
       0.06296472])

In [15]:
#  2020.10.04  晚上1

from sklearn.linear_model import LogisticRegression
# 第二层使用逻辑回归进行stack
LogisticRegression(solver='lbfgs')
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
# 测试集的预测结果
test_pred = stacker.predict()
test_pred

array([0.08456502, 0.34897732, 0.60757832, ..., 0.16541839, 0.20391893,
       0.06514054])

In [15]:
#  2020.10.05  上午

from sklearn.linear_model import LogisticRegression
# 第二层使用逻辑回归进行stack
LogisticRegression(solver='lbfgs')
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
# 测试集的预测结果
test_pred = stacker.predict()
test_pred

array([0.08214523, 0.34288137, 0.59147106, ..., 0.20784624, 0.210794  ,
       0.06366602])

In [35]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_1007_1.csv',index=False)

In [23]:
#  2020.10.01  凌晨

from sklearn.linear_model import LogisticRegression
# 第二层使用逻辑回归进行stack
LogisticRegression(solver='lbfgs')
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
# 测试集的预测结果
test_pred = stacker.predict()
test_pred

array([0.08788547, 0.3337114 , 0.63268724, ..., 0.16219364, 0.21063297,
       0.06451398])

In [24]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_gbdt_1001_1.csv',index=False)

In [16]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_gbdt_1004_1.csv',index=False)

接下来，确认gbdt模型是否可以输入缺失值。。

In [53]:
from sklearn.linear_model import LogisticRegression
# 第二层使用逻辑回归进行stack
LogisticRegression(solver='lbfgs')
stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
# 测试集的预测结果
test_pred = stacker.predict()
test_pred

array([0.08425282, 0.35616588, 0.63669996, ..., 0.21029457, 0.21533899,
       0.06397597])

In [54]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_gbdt_0930_1.csv',index=False)

In [31]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_0929_1.csv',index=False)

In [69]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_0928_1.csv',index=False)

###  使用blending方法进行模型融合

In [55]:
# 构建第一层新特征，将训练集切分成8:2，其中80%用于训练基学习器，20%用于构建新特征
blend_ds = pipeline.blend(proportion=0.2,seed=111)
# 第二层使用逻辑回归进行blend
blender = Classifier(dataset=blend_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
# 测试集的预测结果
test_pred = blender.predict()
test_pred

[14:33:20] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69638	eval-auc:0.69683
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:0.73188	eval-auc:0.72735
[400]	train-auc:0.74155	eval-auc:0.73145
[600]	train-auc:0.74830	eval-auc:0.73299
[800]	train-auc:0.75422	eval-auc:0.73409
[1000]	train-auc:0.75944	eval-auc:0.73465
[1200]	train-auc:0.76435	eval-auc:0.73501
[1400]	train-auc:0.76892	eval-auc:0.73516
[1600]	train-auc:0.77333	eval-auc:0.73518
[1800]	train-auc:0.77747	eval-auc:0.73514
[2000]	train-auc:0.78141	eval-auc:0.73517
Stopping. Best iter

array([0.0824786 , 0.35637054, 0.66376031, ..., 0.17701104, 0.19292704,
       0.06762717])

In [56]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_gbdt_0930_2.csv',index=False)

In [33]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_cat_0929_2.csv',index=False)

In [71]:
sub = pd.DataFrame()
sub['id'] = X_test.index
sub['isDefault'] = test_pred
sub.to_csv('submission_lgbr_xgbr_0928_2.csv',index=False)